In [14]:
import pandas as pd
import praw 
import sqlite3

In [15]:
df = pd.read_csv('/Users/jacksonsorenson/Documents/Computational Media Lab/Original CSV/VAMoS - Reddit Handles _ Verified and Public.csv')
df.head()

,ResponseId,REDDIT
0,R_0369xXP09aPOajv,@dj6eyes
1,R_10VkQQQBDp38cS4,Fenveon11037
2,R_1CfDJYE2y1wwPDd,HowSheGoinEh02
3,R_1dH45I2SxKxpLDA,CursivePluto
4,R_1eEIL6jLKSiCaNq,Yexai_


In [16]:
# Connecting to Reddit API with Praw
reddit = praw.Reddit(
    client_id= "4Fj-6rBvYvhjEOb_3xw6PA",
    client_secret= "t83zQerjf3UZjUMmV5DiPZcuFOtM5g",
    user_agent= "Vaping Reddit Scrape by Jackson Sorenson"   )

In [17]:
# Creating a 'posts' data base to mine all post data from users considtign of their username, the title of their post, the body of their post, created utc, subreddit name, and amount of upvotes(score)

conn = sqlite3.connect('post_data.db')
cursor = conn.cursor()


cursor.execute('''
CREATE TABLE IF NOT EXISTS posts (
    username TEXT,
    title TEXT,
    selftext TEXT,
    created_utc INTEGER,
    subreddit TEXT,
    score INTEGER
)
''')

conn.commit()
conn.close() # Closing the data base each time, had struggle earlier. Will see this in the future code.

In [18]:
#Connecting to data base and mining user post data

conn = sqlite3.connect('post_data.db')
cursor = conn.cursor()

for username in df["REDDIT"]:
    try:
        user = reddit.redditor(username)
        for submission in user.submissions.new(limit=1000):
            print(submission.id, submission.title, submission.selftext, submission.created_utc, submission.subreddit.display_name, submission.score)
            
            cursor.execute('''
                INSERT INTO posts (username, title, selftext, created_utc, subreddit, score)
                VALUES (?, ?, ?, ?, ?, ?)
            ''', (username, submission.title, submission.selftext, submission.created_utc, submission.subreddit.display_name, submission.score))

    except Exception as e:
        print(f"Could not find data for user {username}: {e}")

conn.commit()
conn.close()

Could not find data for user @dj6eyes: received 404 HTTP response
1gk23ju “My Dear Clematis” SGHR Guess who got into Alien Stage back when Round 7 released and immediately thought of dropping these two into the mix. ^ ^ There’s also 4 more images I made but I don’t know this subreddit’s stance on blood in fan art so better safe than sorry. The Florian design I used is by @/pkmhell on Twitter 1730795214.0 KieranDefenseSquad 14
1f0lvpn So I made a Kieran papercraft  1724552135.0 KieranDefenseSquad 34
b3nt5g The Happy Hob The man did it 

[https://clips.twitch.tv/CourteousDignifiedKumquatStoneLightning?tt\_content=chat\_card&tt\_medium=twitch\_chat](https://clips.twitch.tv/CourteousDignifiedKumquatStoneLightning?tt_content=chat_card&tt_medium=twitch_chat) 1553154956.0 darksouls 1334
7wqxnw Console to PC Plz I beg DE I want to play my game in the superior system Plz 1518325846.0 Warframe 0
Could not find data for user Yexai_: received 404 HTTP response
Could not find data for user u/Educat

In [19]:
# Make sure data is being mined and put into 'post' table
conn = sqlite3.connect('post_data.db')
df_posts = pd.read_sql_query("SELECT * FROM posts", conn)
df_posts.head()

,username,title,selftext,created_utc,subreddit,score
0,test_user,Test Title,This is a test post,1234567890,test_subreddit,100
1,Fenveon11037,“My Dear Clematis” SGHR,Guess who got into Alien Stage back when Round...,1730795214,KieranDefenseSquad,14
2,Fenveon11037,So I made a Kieran papercraft,,1724552135,KieranDefenseSquad,34
3,CursivePluto,The Happy Hob,The man did it \n\n[https://clips.twitch.tv/Co...,1553154956,darksouls,1334
4,CursivePluto,Console to PC,Plz I beg DE I want to play my game in the sup...,1518325846,Warframe,0


In [22]:
# Turn int a CSV File
df_posts.to_csv('post_data.csv', index=False)